# Exploratory Data Analysis on PWM Behavioral Data

**Goal** This notebook will explore the new dataset to see what trends are present now that violations are properly tracked and timeouts are not incorrectly marked as violations.

## Load In

## Hit Rates

* over stages by animal (save data out)

## Violation Rates

- over stages by animal (save data out)

## Number of Delays


In [ ]:
# get unique delay values for each animal
unique_delays = (
    RAT_DF.groupby(["subject_id", "training_stage_cat"]).delay.unique().reset_index()
)

# get the number of unique delay values for each animal
unique_delays["n_unique_delays"] = unique_delays.delay.apply(lambda x: len(x))

In [ ]:
g = sns.FacetGrid(
    data=RAT_DF,
    row="subject_id",
    col="training_stage_cat",
    hue="training_stage_cat",
    sharey="col",
    sharex="col",
    height=4,
    aspect=1.3,
)
g.map(sns.histplot, "delay")
g.set_titles("{row_name} stage {col_name}")

In [ ]:
g = sns.catplot(
    x="training_stage_cat",
    y="n_unique_delays",
    col="subject_id",
    col_wrap=4,
    hue="subject_id",
    data=unique_delays.query("training_stage_cat != 1"),
    kind="bar",
    dodge=False,
    sharey=False,
)

## Performance ~ Delay

### Hits

In [ ]:
fig,ax = plt.subplots(figsize=(12, 4))
sns.barplot(data=RAT_DF.query("training_stage >=  3"), x="delay", y="hit", color='green',ax=ax)
_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set(title="Stage 3+ Hit Rates ~ Delay", ylim=(0, 1))

In [ ]:
g = sns.catplot(
    x="delay",
    y="hit",
    col="subject_id",
    col_wrap=4,
    hue="subject_id",
    data=RAT_DF.query("training_stage >=  3"),
    kind="bar",
    dodge=False,
    sharey=True,
    sharex=False,
)

g.set(ylim=(0.5, 1))

### Violations

In [ ]:
discrete_delay_df = RAT_DF.query("training_stage >=  3").groupby(["delay"]).hit.mean().reset_index()
discrete_delay_df['viol'] =  RAT_DF.query("training_stage >=  3").groupby(["delay"]).violation.mean().values
discrete_delay_df.plot(kind="bar", x="delay", y=["hit", "viol"], color=["green", "orange"], figsize=(12, 4))

Next notebook: EDA_02_predict_perf


This requires making a large data frame with 

animal id, n delays, viol stage 1-4, hit stage 1-4
Predict time to train

Predict final hit rate